<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/seed_tts_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/BytedanceSpeech/seed-tts-eval

In [36]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')


In [37]:
!huggingface-cli login --token $HF_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `new` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `new`


In [ ]:
!huggingface-cli download --quiet  Dongchao/detect_dataset --local-dir /datasets/Dongchao/detect_dataset

In [32]:
!huggingface-cli download --help

usage: huggingface-cli <command> [<args>] download [-h]
                                                   [--repo-type {model,dataset,space}]
                                                   [--revision REVISION]
                                                   [--include [INCLUDE ...]]
                                                   [--exclude [EXCLUDE ...]]
                                                   [--cache-dir CACHE_DIR]
                                                   [--local-dir LOCAL_DIR]
                                                   [--local-dir-use-symlinks {auto,True,False}]
                                                   [--force-download]
                                                   [--resume-download]
                                                   [--token TOKEN] [--quiet]
                                                   [--max-workers MAX_WORKERS]
                                                   repo_id [filenames ...]

positional 

In [ ]:
#!huggingface-cli download --repo-type dataset --quiet yuekai/seed_tts_eval --local-dir /datasets/yuekai/seed_tts_eval

In [11]:
!ls -lh /datasets/Dongchao/detect_dataset

total 13G
-rw-r--r-- 1 root root 216M May 17 10:39 Audio_Song_Actors_01-24.zip
-rw-r--r-- 1 root root 199M May 17 10:39 Audio_Speech_Actors_01-24.zip
-rw-r--r-- 1 root root 302M May 17 10:39 gen1.tar.gz
-rw-r--r-- 1 root root 5.4G May 17 10:43 gen2.tar.gz
-rw-r--r-- 1 root root 5.1G May 17 10:43 gen3.tar.gz
-rw-r--r-- 1 root root 1.6M May 17 10:39 new_voice.zip
drwxr-xr-x 4 1000 1000 4.0K Jun  3  2024 seedtts_testset
-rw-r--r-- 1 root root 1.2G May 17 10:39 seedtts_testset.tar


In [ ]:
!tar xvf /datasets/Dongchao/detect_dataset/seedtts_testset.tar -C /datasets/Dongchao/detect_dataset

## Dataset
You can download the test set for all tasks from [this link](https://drive.google.com/file/d/1GlSjVfSHkW3-leKKBlfrjuuTGqQ_xaLP/edit).
The test set is mainly organized using the method of meta file. The meaning of each line in the meta file:

filename | the text of the prompt | the audio of the prompt | the text to be synthesized | the ground truth counterpart corresponding to the text to be synthesized （if exists）.

For different tasks, we adopt different meta files:
* Zero-shot text-to-speech (TTS):
  * EN: en/meta.lst
  * ZH: zh/meta.lst
  * ZH (hard case): zh/hardcase.lst
* Zero-shot voice conversion (VC):
  * EN: en/non_para_reconstruct_meta.lst
  * ZH: zh/non_para_reconstruct_meta.lst


In [17]:
!ls /datasets/Dongchao/detect_dataset/seedtts_testset/en

meta.lst  non_para_reconstruct_meta.lst  prompt-wavs  wavs


In [18]:
!ls -lh /datasets/Dongchao/detect_dataset/seedtts_testset/en | grep -v wav
!ls -lh /datasets/Dongchao/detect_dataset/seedtts_testset/zh | grep -v wav


total 744K
-rw-r--r-- 1 1000 1000 242K Jun  3  2024 meta.lst
-rw-r--r-- 1 1000 1000 329K Jun  3  2024 non_para_reconstruct_meta.lst
total 1.1M
-rw-r--r-- 1 1000 1000 112K Jun  3  2024 hardcase.lst
-rw-r--r-- 1 1000 1000 352K Jun  3  2024 meta.lst
-rw-r--r-- 1 1000 1000 441K Jun  3  2024 non_para_reconstruct_meta.lst


In [24]:
!cat /datasets/Dongchao/detect_dataset/seedtts_testset/en/meta.lst | wc -l
!cat /datasets/Dongchao/detect_dataset/seedtts_testset/en/non_para_reconstruct_meta.lst | wc -l

1088
1086


In [25]:
!cat /datasets/Dongchao/detect_dataset/seedtts_testset/zh/meta.lst | wc -l
!cat /datasets/Dongchao/detect_dataset/seedtts_testset/zh/hardcase.lst | wc -l
!cat /datasets/Dongchao/detect_dataset/seedtts_testset/zh/non_para_reconstruct_meta.lst | wc -l

2020
400
2018


In [26]:
!head /datasets/Dongchao/detect_dataset/seedtts_testset/en/meta.lst
print("---"*20)
!head /datasets/Dongchao/detect_dataset/seedtts_testset/en/non_para_reconstruct_meta.lst

common_voice_en_10119832-common_voice_en_10119840|We asked over twenty different people, and they all said it was his.|prompt-wavs/common_voice_en_10119832.wav|Get the trust fund to the bank early.
common_voice_en_10119832-common_voice_en_10119847|We asked over twenty different people, and they all said it was his.|prompt-wavs/common_voice_en_10119832.wav|The stained glass offered a hypnotic atmosphere.
common_voice_en_103675-common_voice_en_103676|I'm never more aware of a room's acoustics than when I'm trying to enjoy a snack I have no intention of sharing.|prompt-wavs/common_voice_en_103675.wav|One by one, the campfires were extinguished, and the oasis fell as quiet as the desert.
common_voice_en_103675-common_voice_en_103677|I'm never more aware of a room's acoustics than when I'm trying to enjoy a snack I have no intention of sharing.|prompt-wavs/common_voice_en_103675.wav|The boy knew the desert sensed his fear.
common_voice_en_10933823-common_voice_en_10933822|Sometimes I overth

In [28]:
!ls -lh /datasets/Dongchao/detect_dataset/seedtts_testset/en/wavs/ | grep common_voice_en_10119832-common_voice_en_10119840

-rwxr-xr-x 1 1000 1000 117K Jun  3  2024 common_voice_en_10119832-common_voice_en_10119840.wav


In [30]:
from IPython.display import Audio
Audio("/datasets/Dongchao/detect_dataset/seedtts_testset/en/prompt-wavs/common_voice_en_10119832.wav")

In [29]:
from IPython.display import Audio
Audio("/datasets/Dongchao/detect_dataset/seedtts_testset/en/wavs/common_voice_en_10119832-common_voice_en_10119840.wav")

In [39]:
!head /datasets/Dongchao/detect_dataset/seedtts_testset/zh/meta.lst
print("---"*20)
!head /datasets/Dongchao/detect_dataset/seedtts_testset/zh/non_para_reconstruct_meta.lst
print("---"*20)
!head /datasets/Dongchao/detect_dataset/seedtts_testset/zh/hardcase.lst

10002287-00000095|在此奉劝大家别乱打美白针。|prompt-wavs/10002287-00000094.wav|简单地说，这相当于惠普把消费领域市场拱手相让了。
10002287-00000099|在此奉劝大家别乱打美白针。|prompt-wavs/10002287-00000094.wav|可是对另外两台音调失准，怪声怪气的琴，他束手窝策。
10002290-00000102|对此，乔宝云认为有夸大的成分。|prompt-wavs/10002290-00000094.wav|说说你们在京藏高速怀安段，堵车现场看到的情况。
10002290-00000011|对此，乔宝云认为有夸大的成分。|prompt-wavs/10002290-00000094.wav|自动驾驶将大幅提升出行安全，效率。
10002298-00000016|顺风时提高警惕，逆风时笃定前行。|prompt-wavs/10002298-00000001.wav|共同建设面向未来的交通，和出行服务新生态。
10002298-00000019|顺风时提高警惕，逆风时笃定前行。|prompt-wavs/10002298-00000001.wav|我们将为全球城市的可持续发展贡献力量。
10002309-00000033|前方有左急转弯，请减速慢行。|prompt-wavs/10002309-00000029.wav|目前中国互联网国际化还在开拓阶段。
10002309-00000011|前方有左急转弯，请减速慢行。|prompt-wavs/10002309-00000029.wav|自动驾驶将大幅提升出行安全，效率。
10002334-00000041|前方有左急转弯，请减速慢行。|prompt-wavs/10002334-00000029.wav|经济发展是城市高效宜居发展的基石。
10002334-00000043|前方有左急转弯，请减速慢行。|prompt-wavs/10002334-00000029.wav|经济发展的不平不充分问题日益突出。
------------------------------------------------------------
10002287-00000094_10002290-00000102|在此奉劝大家别乱打美白针。|prompt-w

In [40]:
from IPython.display import Audio
Audio("/datasets/Dongchao/detect_dataset/seedtts_testset/zh/prompt-wavs/10002287-00000094.wav")

In [41]:
from IPython.display import Audio
Audio("/datasets/Dongchao/detect_dataset/seedtts_testset/zh/wavs/10002287-00000095.wav")